### Carregamento

In [1]:
import pandas as pd
# Função para carregar os dados
def carregar_dados(caminho_arquivo, coluna_data=None):
    """
    Carrega um arquivo CSV com dados.
    
    Parâmetros:
    - caminho_arquivo (str): Caminho do arquivo CSV.
    - coluna_data (str, opcional): Nome da coluna de datas para converter para datetime.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os dados carregados.
    """
    # Carrega o CSV com separador ';'
    dados = pd.read_csv(caminho_arquivo, sep=',')
    
    # Converte a coluna de datas para datetime com timezone, se especificada
    if coluna_data:
        dados[coluna_data] = pd.to_datetime(dados[coluna_data], utc=True)
    
    return dados
# Caminho
caminho_arquivo = '../data/raw/sales_data.csv'
# Carrega os dados com o separador correto
dados = carregar_dados(caminho_arquivo)

# Exibir as primeiras linhas do DataFrame
dados.head(10)


,Product_ID,Sale_Date,Sales_Rep,Region,Sales_Amount,Quantity_Sold,Product_Category,Unit_Cost,Unit_Price,Customer_Type,Discount,Payment_Method,Sales_Channel,Region_and_Sales_Rep
0,1052,2023-02-03,Bob,North,5053.97,18,Furniture,152.75,267.22,Returning,0.09,Cash,Online,North-Bob
1,1093,2023-04-21,Bob,West,4384.02,17,Furniture,3816.39,4209.44,Returning,0.11,Cash,Retail,West-Bob
2,1015,2023-09-21,David,South,4631.23,30,Food,261.56,371.40,Returning,0.20,Bank Transfer,Retail,South-David
3,1072,2023-08-24,Bob,South,2167.94,39,Clothing,4330.03,4467.75,New,0.02,Credit Card,Retail,South-Bob
4,1061,2023-03-24,Charlie,East,3750.20,13,Electronics,637.37,692.71,New,0.08,Credit Card,Online,East-Charlie
5,1021,2023-02-11,Charlie,West,3761.15,32,Food,900.79,1106.51,New,0.21,Cash,Online,West-Charlie
6,1083,2023-04-11,Bob,West,618.31,29,Furniture,2408.81,2624.09,Returning,0.14,Cash,Online,West-Bob
7,1087,2023-01-06,Eve,South,7698.92,46,Furniture,3702.51,3964.65,New,0.12,Bank Transfer,Online,South-Eve
8,1075,2023-06-29,David,South,4223.39,30,Furniture,738.06,1095.45,New,0.05,Bank Transfer,Online,South-David
9,1075,2023-10-09,Charlie,West,8239.58,18,Clothing,2228.35,2682.34,New,0.13,Bank Transfer,Online,West-Charlie


### Tamanho da base

In [2]:
def verificar_tamanho_base(df):
    """
    Verifica o tamanho da base de dados em termos de número de linhas e colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - dict: Dicionário com o número de linhas e colunas.
    """
    tamanho = {
        'Número de Linhas': df.shape[0],
        'Número de Colunas': df.shape[1]
    }
    
    return tamanho

# Verificar o tamanho da base de dados
tamanho_base = verificar_tamanho_base(dados)
tamanho_base

{'Número de Linhas': 1000, 'Número de Colunas': 14}

### Valores nulos

In [4]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)


### Valores duplicados

In [5]:
def verificar_valores_duplicados(df, colunas=None):
    """
    Verifica valores duplicados em um DataFrame, fornecendo análise detalhada.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado
    - colunas (list, opcional): Lista específica de colunas para verificar duplicatas.
                               Se None, verifica todas as colunas.
    
    Retorna:
    - dict: Dicionário com informações sobre duplicatas para cada coluna analisada
    """
    resultado = {}
    
    # Se nenhuma coluna específica for fornecida, usar todas as colunas
    if colunas is None:
        colunas = df.columns
    
    # Análise de duplicatas por coluna
    for coluna in colunas:
        # Encontrar valores duplicados
        duplicados = df[df[coluna].duplicated(keep='first')]
        qtd_duplicados = len(duplicados)
        
        if qtd_duplicados > 0:
            # Contagem de ocorrências de cada valor duplicado
            valores_duplicados = df[coluna].value_counts()[df[coluna].value_counts() > 1]
            
            resultado[coluna] = {
                'possui_duplicados': True,
                'quantidade_duplicados': qtd_duplicados,
                'valores_e_contagem': valores_duplicados.to_dict()
            }
        else:
            resultado[coluna] = {
                'possui_duplicados': False,
                'quantidade_duplicados': 0,
                'valores_e_contagem': {}
            }
    
    # Exibir resumo
    print("=== Análise de Valores Duplicados ===\n")
    
    for coluna, info in resultado.items():
        print(f"\nColuna: {coluna}")
        if info['possui_duplicados']:
            print(f"Status: Possui {info['quantidade_duplicados']} valores duplicados")
            print("Valores duplicados e suas ocorrências:")
            for valor, contagem in info['valores_e_contagem'].items():
                print(f"  - Valor: {valor} | Ocorrências: {contagem}")
        else:
            print("Status: Não possui valores duplicados")
    
    return resultado

# Executar verificação de valores duplicados
resultados_duplicados = verificar_valores_duplicados(dados)

=== Análise de Valores Duplicados ===


Coluna: Product_ID
Status: Possui 900 valores duplicados
Valores duplicados e suas ocorrências:
  - Valor: 1090 | Ocorrências: 20
  - Valor: 1062 | Ocorrências: 19
  - Valor: 1092 | Ocorrências: 19
  - Valor: 1099 | Ocorrências: 18
  - Valor: 1033 | Ocorrências: 18
  - Valor: 1001 | Ocorrências: 16
  - Valor: 1086 | Ocorrências: 15
  - Valor: 1058 | Ocorrências: 15
  - Valor: 1017 | Ocorrências: 15
  - Valor: 1089 | Ocorrências: 14
  - Valor: 1024 | Ocorrências: 14
  - Valor: 1019 | Ocorrências: 14
  - Valor: 1012 | Ocorrências: 13
  - Valor: 1008 | Ocorrências: 13
  - Valor: 1051 | Ocorrências: 13
  - Valor: 1023 | Ocorrências: 13
  - Valor: 1096 | Ocorrências: 13
  - Valor: 1063 | Ocorrências: 13
  - Valor: 1016 | Ocorrências: 13
  - Valor: 1002 | Ocorrências: 13
  - Valor: 1036 | Ocorrências: 12
  - Valor: 1039 | Ocorrências: 12
  - Valor: 1047 | Ocorrências: 12
  - Valor: 1054 | Ocorrências: 12
  - Valor: 1061 | Ocorrências: 12
  - Valor: 103

### Remoção de espaços

In [6]:
def remover_espacos(df):
    """
    Remove espaços em branco à esquerda e à direita das strings em todas as colunas e nos nomes das colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser processado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os espaços removidos.
    - pandas.DataFrame: DataFrame com informações sobre as remoções realizadas.
    """
    # Verificar se o DataFrame é válido
    if not isinstance(df, pd.DataFrame):
        raise ValueError("O argumento fornecido não é um DataFrame.")
    
    # Verificar espaços nos nomes das colunas antes
    colunas_com_espacos = [col for col in df.columns if col != col.strip()]
    espacos_colunas_antes = len(colunas_com_espacos)
    
    # Remover espaços dos nomes das colunas
    df.columns = df.columns.str.strip()
            
    # Inicializar lista para armazenar informações sobre remoções
    informacoes_remocoes = []

    # Adicionar informação sobre limpeza dos nomes das colunas
    informacoes_remocoes.append({
        'Coluna': 'NOMES DAS COLUNAS',
        'Espaços Antes': espacos_colunas_antes,
        'Espaços Depois': 0,
        'Remoção Bem-Sucedida': True
    })

    # Aplicar strip em todas as colunas de tipo object e category
    for col in df.select_dtypes(include=['object', 'category']).columns:
        # Contar espaços em branco antes da remoção
        espacos_antes = df[col].apply(lambda x: x != x.strip() if isinstance(x, str) else False).sum()
        
        # Remover espaços em branco
        df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)
        
        # Contar espaços em branco após a remoção
        espacos_depois = df[col].apply(lambda x: x != x.strip() if isinstance(x, str) else False).sum()
        
        # Verificar se a remoção foi bem-sucedida
        sucesso = espacos_depois == 0
        
        # Adicionar informações à lista
        informacoes_remocoes.append({
            'Coluna': col,
            'Espaços Antes': espacos_antes,
            'Espaços Depois': espacos_depois,
            'Remoção Bem-Sucedida': sucesso
        })
    
    # Criar DataFrame com as informações de remoção
    df_informacoes_remocoes = pd.DataFrame(informacoes_remocoes)
    
    return df, df_informacoes_remocoes

# Exemplo de uso:
dados, informacoes_remocoes = remover_espacos(dados)

# Exibir as informações sobre as remoções
print("Informações sobre a remoção de espaços em branco:")
display(informacoes_remocoes)

Informações sobre a remoção de espaços em branco:


,Coluna,Espaços Antes,Espaços Depois,Remoção Bem-Sucedida
0,NOMES DAS COLUNAS,0,0,True
1,Sale_Date,0,0,True
2,Sales_Rep,0,0,True
3,Region,0,0,True
4,Product_Category,0,0,True
5,Customer_Type,0,0,True
6,Payment_Method,0,0,True
7,Sales_Channel,0,0,True
8,Region_and_Sales_Rep,0,0,True


### Verificação tipo dos dados

In [8]:
# Função para verificar o tipo dos dados/colunas
def verificar_tipo_dados(df):
    """
    Verifica o tipo dos dados/colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os nomes das colunas e seus respectivos tipos de dados.
    """
    # Obter os tipos de dados das colunas
    tipos_dados = df.dtypes
    
    # Criar um DataFrame com os resultados
    tabela_tipos = pd.DataFrame({
        'Coluna': tipos_dados.index,
        'Tipo de Dado': tipos_dados.values
    })
    
    return tabela_tipos

# Verificar o tipo dos dados/colunas
tabela_tipos = verificar_tipo_dados(dados)
tabela_tipos

,Coluna,Tipo de Dado
0,Product_ID,int64
1,Sale_Date,object
2,Sales_Rep,object
3,Region,object
4,Sales_Amount,float64
5,Quantity_Sold,int64
6,Product_Category,object
7,Unit_Cost,float64
8,Unit_Price,float64
9,Customer_Type,object


### Valores máx/min

In [9]:
def valores_max_min(df):
    """
    Percorre todas as colunas numéricas (int ou float) de um DataFrame e retorna uma tabela com os valores máximos e mínimos de cada coluna.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado.

    Retorna:
    - pandas.DataFrame: DataFrame com os valores máximos e mínimos de cada coluna numérica.
    """
    # Selecionar apenas colunas numéricas
    colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    
    # Criar listas para armazenar os resultados
    colunas = []
    valores_min = []
    valores_max = []
    
    # Percorrer cada coluna numérica e calcular os valores máximo e mínimo
    for coluna in colunas_numericas:
        colunas.append(coluna)
        valores_min.append(df[coluna].min())
        valores_max.append(df[coluna].max())
    
    # Criar o DataFrame final com todos os resultados de uma vez
    tabela_max_min = pd.DataFrame({
        'Coluna': colunas,
        'Valor Mínimo': valores_min,
        'Valor Máximo': valores_max
    })
    
    return tabela_max_min

# Aplicar a função e exibir a tabela de valores máximos e mínimos
tabela_max_min = valores_max_min(dados)
tabela_max_min

,Coluna,Valor Mínimo,Valor Máximo
0,Product_ID,1001.00,1100.00
1,Sales_Amount,100.12,9989.04
2,Quantity_Sold,1.00,49.00
3,Unit_Cost,60.28,4995.30
4,Unit_Price,167.12,5442.15
5,Discount,0.00,0.30


### Conversão otimização

In [10]:
def modificar_tipo_colunas(df, colunas_tipos):
    """
    Modifica o tipo de dado de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_tipos (dict): Dicionário onde as chaves são os nomes das colunas e os valores são os tipos de dados desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os tipos de dados modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_tipos.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Modifica o tipo de dado das colunas especificadas
    for coluna, tipo in colunas_tipos.items():
        try:
            df[coluna] = df[coluna].astype(tipo)
            print(f"✅ Coluna '{coluna}' convertida para '{tipo}' com sucesso.")
        except Exception as e:
            print(f"⚠️ Erro ao converter a coluna '{coluna}' para '{tipo}': {e}")
    
    return df

# Exemplo de uso:
colunas_tipos = {
       # Colunas numéricas
    'Product_ID': 'int32',
    'Sales_Amount': 'float32',
    'Quantity_Sold': 'int8',
    'Unit_Cost': 'float32',
    'Unit_Price': 'float32',
    'Discount': 'float16',
    
    # Colunas categóricas
    'Region': 'category',
    'Product_Category': 'category',
    'Customer_Type': 'category',
    'Payment_Method': 'category',
    'Sales_Channel': 'category',
    
    # Data
    'Sale_Date': 'datetime64[ns]'
  
}
 

dados = modificar_tipo_colunas(dados, colunas_tipos)
dados.dtypes # Verificar os tipos de dados


✅ Coluna 'Product_ID' convertida para 'int32' com sucesso.
✅ Coluna 'Sales_Amount' convertida para 'float32' com sucesso.
✅ Coluna 'Quantity_Sold' convertida para 'int8' com sucesso.
✅ Coluna 'Unit_Cost' convertida para 'float32' com sucesso.
✅ Coluna 'Unit_Price' convertida para 'float32' com sucesso.
✅ Coluna 'Discount' convertida para 'float16' com sucesso.
✅ Coluna 'Region' convertida para 'category' com sucesso.
✅ Coluna 'Product_Category' convertida para 'category' com sucesso.
✅ Coluna 'Customer_Type' convertida para 'category' com sucesso.
✅ Coluna 'Payment_Method' convertida para 'category' com sucesso.
✅ Coluna 'Sales_Channel' convertida para 'category' com sucesso.
✅ Coluna 'Sale_Date' convertida para 'datetime64[ns]' com sucesso.


Product_ID                       int32
Sale_Date               datetime64[ns]
Sales_Rep                       object
Region                        category
Sales_Amount                   float32
Quantity_Sold                     int8
Product_Category              category
Unit_Cost                      float32
Unit_Price                     float32
Customer_Type                 category
Discount                       float16
Payment_Method                category
Sales_Channel                 category
Region_and_Sales_Rep            object
dtype: object

### Valores únicos

In [12]:
def detectar_valores_unicos(df, coluna):
    """
    Detecta valores únicos em uma coluna de um DataFrame e exibe esses valores de forma amigável.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado.
    - coluna (str): Nome da coluna cujos valores únicos serão detectados.

    Retorna:
    - pandas.DataFrame: DataFrame com os valores únicos e suas contagens.
    - str: Mensagem indicando o sucesso da operação e o número de valores únicos encontrados.
    """
    if coluna not in df.columns:
        raise ValueError(f"Coluna '{coluna}' não encontrada no DataFrame.")
    
    # Detectar valores únicos e suas contagens
    valores_unicos = df[coluna].value_counts().reset_index()
    valores_unicos.columns = [coluna, 'Contagem']
    
    # Criar a mensagem de saída
    mensagem = f"Operação bem-sucedida! Foram encontrados {valores_unicos.shape[0]} valores únicos na coluna '{coluna}'."
    
    return valores_unicos, mensagem

# Exemplo de uso:
colunas = [
    'Product_ID', 'Sale_Date', 'Sales_Rep',
    'Region', 'Sales_Amount', 'Quantity_Sold',
    'Product_Category', 'Unit_Cost', 'Unit_Price',
    'Customer_Type', 'Discount', 'Payment_Method',
    'Sales_Channel', 'Region_and_Sales_Rep']

for coluna in colunas:
    valores_unicos, mensagem = detectar_valores_unicos(dados, coluna)
    # Exibir a mensagem e a tabela de valores únicos
    print(mensagem)
    display(valores_unicos)

Operação bem-sucedida! Foram encontrados 100 valores únicos na coluna 'Product_ID'.


,Product_ID,Contagem
0,1090,20
1,1062,19
2,1092,19
3,1099,18
4,1033,18
...,...,...
95,1040,5
96,1011,4
97,1041,4
98,1083,4


Operação bem-sucedida! Foram encontrados 340 valores únicos na coluna 'Sale_Date'.


,Sale_Date,Contagem
0,2023-11-14,8
1,2023-10-20,8
2,2023-10-15,7
3,2023-11-24,7
4,2023-01-12,7
...,...,...
335,2023-05-24,1
336,2023-11-27,1
337,2023-01-19,1
338,2023-01-18,1


Operação bem-sucedida! Foram encontrados 5 valores únicos na coluna 'Sales_Rep'.


...

,Payment_Method,Contagem
0,Credit Card,345
1,Bank Transfer,342
2,Cash,313


Operação bem-sucedida! Foram encontrados 2 valores únicos na coluna 'Sales_Channel'.


,Sales_Channel,Contagem
0,Retail,512
1,Online,488


Operação bem-sucedida! Foram encontrados 20 valores únicos na coluna 'Region_and_Sales_Rep'.


,Region_and_Sales_Rep,Contagem
0,North-Eve,64
1,East-Bob,60
2,East-David,59
3,South-David,56
4,North-David,56
5,North-Charlie,55
6,East-Eve,55
7,West-Bob,54
8,East-Alice,52
9,West-David,51


### Carga

In [13]:
def salvar_dados(df, caminho_arquivo_saida):
    """
    Salva o DataFrame em um arquivo CSV.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser salvo.
    - caminho_arquivo_saida (str): Caminho do arquivo CSV de saída.
    """
    df.to_csv(caminho_arquivo_saida, index=False)
    print(f"✅ Dados salvos com sucesso em '{caminho_arquivo_saida}'")

# Exemplo de uso:
caminho_arquivo_saida = '../data/processed/sales_data_atualizado.csv'
salvar_dados(dados, caminho_arquivo_saida)

✅ Dados salvos com sucesso em '../data/processed/sales_data_atualizado.csv'
